In [1]:
import sys
import numpy as np
import pandas as pd
import os
import cv2
import wandb
from datetime import datetime
from tqdm import tqdm
import argparse
import random
import json

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

from sklearn.metrics import f1_score, accuracy_score

sys.path.append('/home/hong/hc701/HC701-PROJECT')
from VAL import test

In [2]:
from hc701fed.dataset.dataset_list_transform import (
    MESSIDOR_binary_pairs_train,
    MESSIDOR_binary_pairs_test,
    MESSIDOR_binary_Etienne_train,
    MESSIDOR_binary_Etienne_test,
    MESSIDOR_binary_Brest_train,
    MESSIDOR_binary_Brest_test
)

centerlized_train = ConcatDataset([MESSIDOR_binary_pairs_train, MESSIDOR_binary_Etienne_train, MESSIDOR_binary_Brest_train])
centerlized_test = ConcatDataset([MESSIDOR_binary_pairs_test, MESSIDOR_binary_Etienne_test, MESSIDOR_binary_Brest_train])

horizontal_flip {'p': 0.5}
random_rotation {'degrees': [-10, 10]}


In [3]:
import timm
from timm.models import create_model

batch_size = 32
num_epochs = 10

# use resnet18 as the base model
model = create_model('resnet50', pretrained=True, num_classes=2)

# use the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# define the loss function
criterion = torch.nn.CrossEntropyLoss()

# define the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Data loader
train_loader_pairs = DataLoader(MESSIDOR_binary_pairs_train, batch_size=batch_size, shuffle=True)
train_loader_Etienne = DataLoader(MESSIDOR_binary_Etienne_train, batch_size=batch_size, shuffle=True)
train_loader_Brest = DataLoader(MESSIDOR_binary_Brest_train, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(centerlized_train, batch_size=batch_size, shuffle=True)

test_loader_pairs = DataLoader(MESSIDOR_binary_pairs_test, batch_size=batch_size, shuffle=True)
test_loader_Etienne = DataLoader(MESSIDOR_binary_Etienne_test, batch_size=batch_size, shuffle=True)
test_loader_Brest = DataLoader(MESSIDOR_binary_Brest_test, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(centerlized_test, batch_size=batch_size, shuffle=True)

# Define the scheduler make the learning rate linear decay to zero
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: 1 - epoch / num_epochs)

# train the model
for epoch in tqdm(range(num_epochs)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
            
    # Test the model
    # We need f1_score and accuracy_score
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        pred_labels = []
        true_labels = []
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            pred_labels.append(predicted.cpu().numpy().tolist()[0])
            true_labels.append(labels.cpu().numpy().tolist()[0])
        f1 = f1_score(true_labels, pred_labels)
        acc = accuracy_score(true_labels, pred_labels)

        # print('Test Accuracy of the model on the test images: {} %'.format(100 * acc))
        # print('Test F1 of the model on the test images: {}'.format(f1))

        print((acc+f1)/2, 'this is the average of acc and f1')


  0%|          | 0/10 [00:00<?, ?it/s]/home/hong/.conda/envs/py38/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py:184: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor(batch)
 10%|█         | 1/10 [00:04<00:40,  4.45s/it]

0.27083333333333337 this is the average of acc and f1


 20%|██        | 2/10 [00:06<00:23,  3.00s/it]

0.1875 this is the average of acc and f1


 30%|███       | 3/10 [00:08<00:18,  2.65s/it]

...

0.25 this is the average of acc and f1


 90%|█████████ | 9/10 [00:21<00:02,  2.26s/it]

0.3125 this is the average of acc and f1


100%|██████████| 10/10 [00:24<00:00,  2.43s/it]

0.3125 this is the average of acc and f1
